In [ ]:
from meteostat import Monthly
from datetime import datetime
import pandas as pd

# Read excel 

excel_path = r"C:/Projects/Capstone/CO2-Emissions-and-Global-Temperature-Analysis/Data/Cleaned/Avg_Yearly_Temps.xlsx"

df_long = pd.read_excel(excel_path)

cities = ["Rio De Janeiro", "Paris", "Sydney", "Houston"]          # spellings must match Excel
df_excel = (df_long
            .pivot(index="Year", columns="City", values="AvgTemp")
            .loc[:, cities]                             # keep only the four cities
            .round(2))

print("Excel wide table (head):")
print(df_excel.head(), "\n")

# Build Meteostat series

def yearly_means(station_id, start, end):
    m = Monthly(station_id, start, end).fetch()
    m = m.dropna(subset=['tavg'])
    m['year'] = m.index.year
    return (m.groupby('year')['tavg']
              .mean()
              .rename(station_id))

station_ids = {"Rio De Janeiro":     "83746",   # Galeão
               "Paris":   "07156",   # Parc-Montsouris
               "Sydney":  "94767",   # Kingsford-Smith
               "Houston": "72243"}   # Bush-IAH

start, end = datetime(2000, 1, 1), datetime(2022, 12, 31)

df_meteostat = pd.concat(
    {city: yearly_means(sid, start, end) for city, sid in station_ids.items()},
    axis=1
).round(2)

print("Meteostat table (head):")
print(df_meteostat.head(), "\n")

# Align sources

years_common = df_excel.index.intersection(df_meteostat.index)
df_excel     = df_excel.loc[years_common]
df_meteostat = df_meteostat.loc[years_common]

combined = (df_excel.add_suffix(" Excel")
            .join(df_meteostat.add_suffix(" Meteostat")))

for city in cities:
    combined[f"{city} Δ"] = (combined[f"{city} Meteostat"]
                             - combined[f"{city} Excel"]).round(2)

print("── Year-by-year comparison ──")
print(combined); print()

diff_cols   = [f"{c} Δ" for c in cities]
mad_city    = combined[diff_cols].mean().round(2)
mad_overall = combined[diff_cols].stack().mean().round(2)

print("Mean |Δ| by city (°C):")
print(mad_city)
print(f"\nOverall (all {len(years_common)*len(cities)} city-years): {mad_overall} °C")



Excel wide table (head):
City  Rio De Janeiro  Paris  Sydney  Houston
Year                                        
1744             NaN  11.48     NaN      NaN
1745             NaN   4.73     NaN      NaN
1750             NaN  11.18     NaN      NaN
1751             NaN  11.15     NaN      NaN
1752             NaN   6.97     NaN      NaN 

Meteostat table (head):
      Rio De Janeiro  Paris  Sydney  Houston
year                                        
2000           23.93   <NA>   18.21    20.96
2001           24.44   <NA>   18.42    20.33
2002           24.58   <NA>   18.35    20.73
2003           24.58   <NA>   18.18     20.9
2004           23.99   <NA>   18.48    21.38 

── Year-by-year comparison ──
      Rio De Janeiro Excel  Paris Excel  Sydney Excel  Houston Excel  \
2000                 24.36        11.74         17.57          21.34   
2001                 24.99        11.37         17.85          20.64   
2002                 25.18        11.87         17.73          20.67   

In [2]:
from meteostat import Monthly
from datetime import datetime
import pandas as pd

def yearly_means(station_id, start, end):
    """Return a Pandas Series of yearly-mean temperatures (°C) for one station."""
    m = Monthly(station_id, start, end).fetch()          # monthly rows
    m = m.dropna(subset=['tavg'])                       # keep months that have a mean
    m['year'] = m.index.year
    return (m.groupby('year')['tavg']
             .mean()
             .rename(station_id))

start = datetime(2000, 1, 1)
end   = datetime(2022, 12, 31)

rio     = yearly_means('83746', start, end)
paris   = yearly_means('07156', start, end)
sydney  = yearly_means('94767', start, end)
houston = yearly_means('72243', start, end)

table = pd.concat([rio, paris, sydney, houston], axis=1)
table.columns = ['Rio (°C)', 'Paris (°C)', 'Sydney (°C)', 'Houston (°C)']
print(table.round(2))

      Rio (°C)  Paris (°C)  Sydney (°C)  Houston (°C)
year                                                 
2000     23.93        <NA>        18.21         20.96
2001     24.44        <NA>        18.42         20.33
2002     24.58        <NA>        18.35         20.73
2003     24.58        <NA>        18.18          20.9
2004     23.99        <NA>        18.48         21.38
2005     24.59       12.16        18.74         21.31
2006     24.43       11.72        18.42         21.58
2007     24.42       12.43        18.59         21.15
2008     23.57       13.06        17.91          21.2
2009     24.13       12.76        18.76         21.38
2010     24.28       11.17        18.57         20.82
2011     23.96       12.61         18.5         22.06
2012     25.05       11.92        18.61         22.28
2013     24.64       11.92        19.52         20.83
2014      25.3       13.96        19.38         20.42
2015     25.22       13.31        19.18         21.26
2016     24.46       12.55  